<a href="https://colab.research.google.com/github/domschl/tensor-poet/blob/master/eager_poet_with_indie_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eager Tensor Poet

In [1]:
!pip install -U ml-indie-tools

In [1]:
import logging
import os
import sys
import json
import time
import datetime

import numpy as np
import tensorflow as tf

In [2]:
from ml_indie_tools.env_tools import MLEnv
from ml_indie_tools.Gutenberg_Dataset import Gutenberg_Dataset
from ml_indie_tools.Text_Dataset import Text_Dataset

## Preliminary

A tensorflow deep LSTM model for text generation

This code can use either CPU, GPU, TPU when running on Google Colab.

Select the corresponding runtime (menu: **`Runtime / Change runtime type`**)

## 0. Environment

In [3]:
ml_env = MLEnv(platform='tf', accelerator='fastest')
ml_env.describe()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.75.163.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.75.163.138:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


'OS: Linux, Python: 3.7.12, Colab Jupyter Notebook Tensorflow: 2.7.0, TPU: TPU, 8 nodes v2 (8GB)'

In [4]:
if ml_env.is_tpu is True:   # DUPLICATE with LIB!
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # tpu=TPU_ADDRESS)
    # tf.config.experimental_connect_to_cluster(cluster_resolver) # host(cluster_resolver.master())
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)    
    tpu_is_init=True
    tf.compat.v1.disable_eager_execution()
    use_eager=False
else:
    use_eager=True

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.75.163.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.75.163.138:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [5]:
project_name='women_writers'
model_name='lstm_v1'
root_path, project_path, model_path, data_path, log_path = ml_env.init_paths(project_name=project_name, model_name=model_name)

##  1. Text library

`Text_Dataset` and `Gutenberg_Dataset` classes: libraries for training, 
encoding, batch generation, and formatted source display. It read some 
books from Project Gutenberg and supports creation of training batches. 
The output functions support highlighting to allow to compare generated 
texts with the actual sources to help to identify identical (memorized) 
parts.

In [6]:
use_dark_mode=False  # Set to false for white background

In [7]:
logging.basicConfig(level=logging.INFO)
cache_dir = os.path.join(data_path, 'gutenberg_cache')
gd = Gutenberg_Dataset(cache_dir=cache_dir)

In [8]:
# sample searches
search_spec= {"author": ["brontë","Jane Austen", "Virginia Woolf"], "language": ["english"]}

book_list=gd.search(search_spec)
book_cnt = len(book_list)
print(f"{book_cnt} matching books found with search {search_spec}.")
if book_cnt<40:
    # Note: please verify that book_cnt is 'reasonable'. If you plan to use a large number of texts, 
    # consider [mirroring Gutenberg](https://github.com/domschl/ml-indie-tools#working-with-a-local-mirror-of-project-gutenberg)
    book_list = gd.insert_book_texts(book_list, download_count_limit=book_cnt)  
else:
    logging.error("Please verify your book_list, a large number of books is scheduled for download. ABORTED.")

25 matching books found with search {'author': ['brontë', 'Jane Austen', 'Virginia Woolf'], 'language': ['english']}.


In [9]:
td = Text_Dataset(book_list)

In [10]:
def get_random_sample_batch(td, batch_size, length):
    for i in range(batch_size):
        Xi, yi = td.get_random_char_tokenized_sample_pair(length)
        if i==0:
            smpX=np.array(Xi, dtype=np.float32)
            smpy=np.array(yi, dtype=np.float32)
        else:
            smpX = np.vstack((smpX, np.array(Xi, dtype=np.float32)))
            smpy = np.vstack((smpy, np.array(yi, dtype=np.float32)))
    return np.array(smpX), np.array(smpy)

def get_random_onehot_sample_batch(td, batch_size, length):
    X, y = get_random_sample_batch(td, batch_size, length)
    xoh = tf.keras.backend.one_hot(X, len(td.i2c))
    yk = tf.keras.backend.constant(y)
    return xoh, yk

## 2. Use tf.data for texts

In [11]:
SEQUENCE_LEN = 80
iNumBatches = 64
if ml_env.is_tpu is True:
    BATCH_SIZE=768
    use_tpu_model_for_tpu=True
    STATEFUL=False
    LSTM_UNITS = 512
    LSTM_LAYERS = 4
else:
    BATCH_SIZE = 128
    STATEFUL = True
    LSTM_UNITS = 512
    LSTM_LAYERS = 4

if iNumBatches==0:   # XXX WRONG NUMBER for tests!
    NUM_BATCHES=BATCH_SIZE  # int(textlib.total_size/BATCH_SIZE/SEQUENCE_LEN)
else:
    NUM_BATCHES=iNumBatches

In [12]:
dx=[]
dy=[]
for i in range(NUM_BATCHES):
    x,y=get_random_onehot_sample_batch(td, BATCH_SIZE,SEQUENCE_LEN)
    dx.append(x)
    dy.append(y)

In [13]:
data_xy=(dx,dy) # tf.keras.backend.constant(np.array([dx,dy]))

In [14]:
textlib_dataset=tf.data.Dataset.from_tensor_slices(data_xy)

In [15]:
shuffle_buffer=10000
dataset=textlib_dataset.shuffle(shuffle_buffer)  
dataset.take(1)

<TakeDataset shapes: ((768, 80, 188), (768, 80)), types: (tf.float32, tf.float32)>

In [16]:
def build_model(vocab_size, steps, lstm_units, lstm_layers, batch_size, stateful=True):
    model = tf.keras.Sequential([
        # tf.keras.layers.Embedding(vocab_size, embedding_dim,
        #                          batch_input_shape=[batch_size, None]),
        # tf.keras.layers.Flatten(),
        *[tf.keras.layers.LSTM(lstm_units,
                            # input_shape=(timesteps, data_dim)
                            batch_input_shape=[batch_size, steps, vocab_size],
                            return_sequences=True,
                            stateful=stateful,
                            recurrent_initializer='glorot_uniform')  for _ in range(lstm_layers)],
        # *[tf.keras.layers.LSTM(lstm_units,
        #                     return_sequences=True,
        #                     stateful=stateful,
        #                     recurrent_initializer='glorot_uniform') for _ in range(lstm_layers-1)],
        tf.keras.layers.Dense(vocab_size)
        ])
    return model

def build_tpu_model(vocab_size, steps, lstm_units, lstm_layers, batch_size, stateful=True):
    # print("NOT ADAPTED!")
    # with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    #     embedded = tf.keras.layers.Embedding(vocab_size, embedding_dim, embeddings_initializer='uniform', batch_input_shape=[batch_size, None, SEQUENCE_LEN])
    with tpu_strategy.scope():
        lstm = [tf.keras.layers.LSTM(lstm_units,
                        batch_input_shape=[batch_size, steps, vocab_size],
                        return_sequences=True,
                        stateful=stateful,
                        recurrent_initializer='glorot_uniform', unroll=True) for _ in range(lstm_layers)]
#     tf.keras.layers.LSTM(lstm_units,
#                          return_sequences=True,
#                          stateful=stateful,
#                          # recurrent_initializer='glorot_uniform',
#                         unroll=True)
    dense = tf.keras.layers.Dense(vocab_size)
    
    model = tf.keras.Sequential([
        # embedded,
        *lstm,
        dense
        ])
    return model

In [17]:
if ml_env.is_tpu is True:
    if use_tpu_model_for_tpu is True:
        print("tpu, simple model")
        # with tpu_strategy.scope():
        model = build_tpu_model(
          vocab_size = len(td.i2c),
          # embedding_dim=EMBEDDING_DIM,
          steps=SEQUENCE_LEN,
          lstm_units=LSTM_UNITS,
          lstm_layers=LSTM_LAYERS,
          batch_size=BATCH_SIZE,
          stateful=STATEFUL)
    else:
        print("tpu, default model")
        with tpu_strategy.scope():
            model = build_model(
              vocab_size = len(td.i2c),
              steps=SEQUENCE_LEN,
              # embedding_dim=EMBEDDING_DIM,
              lstm_units=LSTM_UNITS,
              lstm_layers=LSTM_LAYERS,
              batch_size=BATCH_SIZE,
              stateful=STATEFUL)        
else:
    print("non-tpu mode")
    model = build_model(
        vocab_size = len(td.i2c),
        # embedding_dim=EMBEDDING_DIM,
        steps=SEQUENCE_LEN,
        lstm_units=LSTM_UNITS,
        lstm_layers=LSTM_LAYERS,
        batch_size=BATCH_SIZE,
        stateful=STATEFUL)

tpu, simple model


### Some sanity checks of the (untrained!) model

In [18]:
if use_eager is True:
    dataset.take(1)

In [19]:
if ml_env.is_tpu is False:  # no sanity for TPU, since eager not supported:
    for input_example_batch, target_example_batch in dataset.take(1):
        model.reset_states()
        example_batch_predictions = model.predict(input_example_batch, batch_size=256)
        print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (768, 80, 512)            1435648   
                                                                 
 lstm_1 (LSTM)               (768, 80, 512)            2099200   
                                                                 
 lstm_2 (LSTM)               (768, 80, 512)            2099200   
                                                                 
 lstm_3 (LSTM)               (768, 80, 512)            2099200   
                                                                 
 dense (Dense)               (768, 80, 188)            96444     
                                                                 
Total params: 7,829,692
Trainable params: 7,829,692
Non-trainable params: 0
_________________________________________________________________


In [21]:
if use_eager is True:
    dataset.take(1)

In [22]:
if ml_env.is_tpu is False:
    sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
    print(sampled_indices)

In [23]:
if ml_env.is_tpu is False:
    print(td.decode(sampled_indices, tokenizer='char'))

### Loss function, optimizer, tensorboard output

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

if ml_env.is_tpu is False:
    example_batch_loss  = loss(target_example_batch, example_batch_predictions)
    print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
    print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [25]:
opti = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.3)
# opti = tf.keras.optimizers.Adam(clipvalue=0.5)
# opti=tf.keras.optimizers.SGD(lr=0.003)

def scalar_loss(labels, logits):
    bl=loss(labels, logits)
    return tf.reduce_mean(bl)

model.compile(optimizer=opti, loss=loss, metrics=[scalar_loss])

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = os.path.join(model_path, 'training_checkpoints')
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

logdir = os.path.join(log_path, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='batch') # , histogram_freq=1) # update_freq='epoch', 

In [27]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 876), started 0:06:52 ago. (Use '!kill 876' to kill it.)

<IPython.core.display.Javascript object>

## The actual training

In [28]:
EPOCHS=20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, tensorboard_callback])

Train on 64 steps
Epoch 1/20
15/64 [======>.......................] - ETA: 6:47 - batch: 7.0000 - size: 1.0000 - loss: 3.7271 - scalar_loss: 3.7271

In [ ]:
# Generate

In [ ]:
use_tpu_for_generation=False

In [ ]:
if not use_tpu_for_generation:
    gen_model = build_model(vocab_size = len(textlib.i2c),
        # embedding_dim=EMBEDDING_DIM,
        steps=SEQUENCE_LEN,
        lstm_units=LSTM_UNITS,
        lstm_layers=LSTM_LAYERS,
        batch_size=1)
else:
    gen_model = build_tpu_model(
          vocab_size = len(textlib.i2c),
          #embedding_dim=EMBEDDING_DIM,
          steps=SEQUENCE_LEN,
          lstm_units=LSTM_UNITS,
          lstm_layers=LSTM_LAYERS,
          batch_size=1,
          stateful=STATEFUL)  # TPUs can't handle stateful=True, and that's deadly for text generation.

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
gen_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
gen_model.build(tf.TensorShape([1, None]))

In [ ]:
gen_model.summary()

In [ ]:
def generate_text_with_tpu(model, start_string, temp=0.6):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 128

  # Converting our start string to numbers (vectorizing)
  cutstr=start_string[-SEQUENCE_LEN:]  # Tpus need the whole history of exactly secuence_len chars, not less, not more.
  input_eval = [textlib.c2i[s] for s in cutstr]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []
  ids=[]

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temp

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_tensor = tf.random.categorical(predictions, num_samples=1)[-1,0]
      if not use_tpu:
          predicted_id=predicted_tensor.numpy()
      else:
          predicted_id=predicted_tensor.eval()
      ids.append(predicted_id)

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(textlib.i2c[predicted_id])
      print("out:"+''.join(text_generated))

      cutstr=(start_string+''.join(text_generated))[-SEQUENCE_LEN:]  # Restore the entire history if last SEQUENCE_LEN chars, to be "stateless"
      input_eval = [textlib.c2i[s] for s in cutstr]
      input_eval = tf.expand_dims(input_eval, 0)

  return (start_string + ''.join(text_generated), ids)

def generate_text(model, start_string, temp=0.6):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 128

  # Converting our start string to numbers (vectorizing)
  cutstr=start_string[0:SEQUENCE_LEN] # 
  input_eval_0 = [textlib.c2i[s] for s in cutstr]
  input_eval_1 = tf.expand_dims(input_eval_0, 0)

  input_eval = tf.keras.backend.one_hot(input_eval_1, len(textlib.i2c))

  # Empty string to store our results
  text_generated = cutstr # []
  ids=input_eval_0 # []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temp

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model.predict(input_eval, steps=1, batch_size=1)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_tensor = tf.random.categorical(predictions, num_samples=1)[-1,0]
      if use_eager is True:
          predicted_id=predicted_tensor.numpy()
      else:
          predicted_id=tf.keras.backend.eval(predicted_tensor)
          print(predicted_id)
      ids.append(predicted_id)

      text_generated +=textlib.i2c[predicted_id]
      text_generated = text_generated[-SEQUENCE_LEN:]
      print(text_generated)

      # input_eval = tf.keras.backend.one_hot(input_eval_1, len(textlib.i2c))
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval_1 = tf.expand_dims(ids[-SEQUENCE_LEN:], 0)
      input_eval = tf.keras.backend.one_hot(input_eval_1, len(textlib.i2c))    
  return (''.join(text_generated), ids)

In [ ]:
start_string="With the clarity of thought of an artificial life form, the discussion went on:"
len(start_string[0:SEQUENCE_LEN])

In [ ]:
if use_tpu_for_generation:
    sess=tf.compat.v1.keras.backend.get_session() # tf.compat.v1.get_default_session()
    with sess.as_default():
        tx,id=generate_text(gen_model, start_string="With the clarity of thought of an artificial life form, the discussion went on:", temp=0.8)
else:
    if use_eager:
        tf.compat.v1.enable_eager_execution()
        if not tf.executing_eagerly():
            print("Eager engine stall.")
        else:
            sess=tf.compat.v1.keras.backend.get_session()
    # with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):  # Speed is about same gpu/cpu
    tx,id=generate_text(gen_model, start_string="With the clarity of thought of an artificial life form, the discussion went on:", temp=0.8)
    print(tx)

In [ ]:
def detectPlagiarism(tx, textlibrary, minQuoteLength=10):
    textlibrary.source_highlight(tx, minQuoteLength)

In [ ]:
txt=textlib.decode(id)
txti=txt.split('\r\n')
for t in txti:
    print(t)

In [ ]:
detectPlagiarism(tx, textlib)

## References:
* <https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/text/text_generation.ipynb>
* <https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpu_and_keras.ipynb>

## 6. A dialog with the trained model [not ported yet]

In [ ]:
# Do a dialog with the recursive neural net trained above:
# def genDialogAnswer(prompt, g_state=None, endPrompt='.', maxEndPrompts=2,
# maxAnswerSize=512, temperature=1.0):


def doDialog():
    # 0.1 (frozen character) - 1.3 (creative/chaotic character)
    temperature = 0.6
    endPrompt = '.'  # the endPrompt character is the end-mark in answers.
    # look for number of maxEndPrompts until answer is finished.
    maxEndPrompts = 4
    maxAnswerSize = 2048  # Maximum length of the answer
    minAnswerSize = 64  # Minimum length of the answer

    with tf.Session() as sess:
        print("Please enter some dialog.")
        print("The net will answer according to your input.")
        print("'bye' for end,")
        print("'reset' to reset the conversation context,")
        print("'temperature=<float>' [0.1(frozen)-1.0(creative)]")
        print("    to change character of the dialog.")
        print("    Current temperature={}.".format(temperature))
        print()
        xso = None
        bye = False
        model.init.run()

        tflogdir = os.path.realpath(model.logdir)
        if not os.path.exists(tflogdir):
            print("You haven't trained a model, no data found at: {}".format(
                trainParams["logdir"]))
            return

        # Used for saving the training parameters periodically
        saver = tf.train.Saver()
        checkpoint_file = os.path.join(tflogdir, model.checkpoint)

        lastSave = tf.train.latest_checkpoint(tflogdir, latest_filename=None)
        if lastSave is not None:
            pt = lastSave.rfind('-')
            if pt != -1:
                pt += 1
                start_iter = int(lastSave[pt:])
            # print("Restoring checkpoint at {}: {}".format(start_iter, lastSave))
            saver.restore(sess, lastSave)
        else:
            print("No checkpoints have been saved at:{}".format(tflogdir))
            return

        # g_state = sess.run([model.init_state_0], feed_dict={model.batch_size: 1})
        doini = True

        bye = False
        while not bye:
            print("> ", end="")
            prompt = input()
            if prompt == 'bye':
                bye = True
                print("Good bye!")
                continue
            if prompt == 'reset':
                doini = True
                # g_state = sess.run([model.init_state_0], feed_dict={model.batch_size: 1})
                print("(conversation context marked for reset)")
                continue
            if prompt[:len("temperature=")] == "temperature=":
                t = float(prompt[len("temperature="):])
                if t > 0.05 and t < 1.4:
                    temperature = t
                    print("(generator temperature now {})".format(t))
                    print()
                    continue
                print("Invalid temperature-value ignored! [0.1-1.0]")
                continue
            xs = ' ' * model.steps
            xso = ''
            for rep in range(1):
                for i in range(len(prompt)):
                    xs = xs[1:]+prompt[i]
                    X_new = np.transpose([[textlib.c2i[sj]] for sj in xs])
                    if doini:
                        doini = False
                        g_state = sess.run(
                            [model.init_state_0], feed_dict={model.X: X_new})
                    g_state, y_pred = sess.run([model.final_state, model.output_softmax_temp],
                                               feed_dict={model.X: X_new, model.init_state: g_state,
                                                          model.temperature: temperature})
            ans = 0
            numEndPrompts = 0
            while (ans < maxAnswerSize and numEndPrompts < maxEndPrompts) or ans < minAnswerSize:

                X_new = np.transpose([[textlib.c2i[sj]] for sj in xs])
                g_state, y_pred = sess.run([model.final_state, model.output_softmax_temp],
                                           feed_dict={model.X: X_new, model.init_state: g_state,
                                                      model.temperature: temperature})
                inds = list(range(model.vocab_size))
                ind = np.random.choice(inds, p=y_pred[0, -1].ravel())
                nc = textlib.i2c[ind]
                if nc == endPrompt:
                    numEndPrompts += 1
                xso += nc
                xs = xs[1:]+nc
                ans += 1
            print(xso.replace("\\n", "\n"))
            textlib.source_highlight(xso, 13)
    return

In [ ]:
# Talk to the net!
doDialog()